In [1]:
import sys
import os
import random
import copy
import time
from pathlib import Path
import hashlib as hl
import pickle
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
import somhos.methods.useful as mu

In [2]:
#data_path = "resources/aminer/v1"
data_path = "../../src/somhos/resources/aminer/v9beta"

In [3]:
tmp_data_path = data_path + "/tmp-pickles"
if not Path(tmp_data_path).exists():
    os.mkdir(tmp_data_path)
kps_tmp_counts = tmp_data_path + "/kps-tmp-counts-%d.pkl"
kps_tmp_keyphrases = tmp_data_path + "/kps-tmp-keyphrases-%d.pkl"
kps_tmp_idocs = tmp_data_path + "/kps-tmp-idocs-%d.pkl"

In [3]:
docs_tmp_train = data_path + "/train-dataset-acm0%d.pkl"
docs_tmp_test = data_path + "/test-dataset-acm0%d.pkl"

In [4]:
kps_file_segment = 150
kps_hashes_counts = {}
kps_hashes_keyphrases = {}
kps_hashes_idocs = {}
for i in range(0, kps_file_segment + 1):
    if Path(kps_tmp_counts % i).exists():
        kps_hashes_counts.update(mu.load_pickle(kps_tmp_counts % i))
        kps_hashes_keyphrases.update(mu.load_pickle(kps_tmp_keyphrases % i))
        kps_hashes_idocs.update(mu.load_pickle(kps_tmp_idocs % i))

In [5]:
test_dataset = []
train_dataset = []
# acm
for i in range(1, 4):
    if Path(docs_tmp_train % i).exists():
        print(docs_tmp_train % i)
        train_dataset += mu.load_pickle(docs_tmp_train % i)
        test_dataset += mu.load_pickle(docs_tmp_test % i)

../../src/somhos/resources/aminer/v9beta/train-dataset-acm01.pkl
../../src/somhos/resources/aminer/v9beta/train-dataset-acm02.pkl
../../src/somhos/resources/aminer/v9beta/train-dataset-acm03.pkl


In [6]:
print(" - Tmp files: %d" % (kps_file_segment))
print(" - Test dataset: %d" % len(test_dataset))
print(" - Train dataset: %d" % len(train_dataset))
# print(" - Keyphrases: %d" % kps_count)
print(" - Keyphrases hashes-counts: %d" % len(kps_hashes_counts))
print(" - Keyphrases normalized hashes: %d" % len(kps_hashes_keyphrases))
print(" - Keyphrases-docsids: %d" % len(kps_hashes_idocs))

 - Tmp files: 150
 - Test dataset: 243520
 - Train dataset: 2146777
 - Keyphrases hashes-counts: 3922202
 - Keyphrases normalized hashes: 3922202
 - Keyphrases-docsids: 3922202


In [7]:
kps_directory = {}
kps_normalized = []
kps_counts = []
kps_docs_counts = []
kps_docs_ids = []
for i, (k, v) in enumerate(sorted(kps_hashes_counts.items(), key=lambda x: x[1], reverse=True)):
    kps_normalized.append(kps_hashes_keyphrases[k])
    kps_counts.append(kps_hashes_counts[k])
    kps_docs_counts.append(len(kps_hashes_idocs[k]))
    kps_docs_ids.append(kps_hashes_idocs[k])
    kps_directory[k] = i

In [8]:
kps_directory_inverse = {}
for k, v in kps_directory.items():
    for idoc in kps_hashes_idocs[k]: 
        kps_directory_inverse.setdefault(idoc, set())
        kps_directory_inverse[idoc].add(v)

In [9]:
print("%d " % len(kps_normalized))
print("%d " % len(kps_counts))
print("%d " % len(kps_docs_counts))
print("%d " % len(kps_docs_ids))
print("%d " % len(kps_directory))
print("%d " % len(kps_directory_inverse))

In [8]:
print("%d " % len(test_dataset))
print("%d " % len(train_dataset))
print("2385066 == %d" % (len(train_dataset) + len(test_dataset)))

243520 
2146777 
2385066 == 2390297


In [13]:
seg_start = 100
seg_end = seg_start + 5
print(kps_normalized[seg_start:seg_end])
print(kps_counts[seg_start:seg_end])
print(kps_docs_counts[seg_start:seg_end])
# print(kps_docs_ids[seg_start:seg_end])
print([kps_directory[mu.hash_16bytes(kn)] for kn in kps_normalized[seg_start:seg_end]])
print([di for i, di in enumerate(kps_directory_inverse.items()) if i < 10])

[b'literature', b'call', b'example', b'n', b'q']
[423, 422, 420, 420, 418]
[396, 418, 395, 256, 184]
[100, 101, 102, 103, 104]
[('index2358587', {0, 769, 1184, 99, 2465, 2824, 11209, 2602, 13291, 3906536, 14, 16722, 3962}), ('index2335651', {0, 159904, 642, 64461, 16, 22, 121, 1147, 39964}), ('index2367736', {0, 8425, 3979, 169422, 324242, 3909106, 127}), ('index2351487', {0, 3918852, 38, 103, 84843, 27628, 14, 16, 17, 80, 5459, 20, 117, 22}), ('index2355729', {0, 529, 2101, 13115, 892}), ('index2366313', {0, 6464, 526, 17, 529, 20, 21, 3898358, 2231, 952, 3899060, 3899773}), ('index2355310', {0, 1, 484, 1956, 25128, 42, 80, 17, 3600, 4530, 20, 229456, 229841, 560372, 1058362, 716570}), ('index2341375', {0, 193, 352, 1800, 1066219, 10189}), ('index2336240', {0, 602048, 3, 1092, 8, 2120, 17, 53, 158}), ('index2355730', {0, 739, 1956, 2307, 1367278, 603, 158})]


In [14]:
kps_normalized_path = data_path + "/kps-normalized-simpseq10-nopost.pkl"
if not Path(kps_normalized_path).exists():
    with open(kps_normalized_path, "wb") as fout:
        pickle.dump(kps_normalized, fout, pickle.HIGHEST_PROTOCOL)
        
kps_counts_path = data_path + "/kps-counts-simpseq10-nopost.pkl"
if not Path(kps_counts_path).exists():
    with open(kps_counts_path, "wb") as fout:
        pickle.dump(kps_counts, fout, pickle.HIGHEST_PROTOCOL)

kps_docs_counts_path = data_path + "/kps-docs-counts-simpseq10-nopost.pkl"
if not Path(kps_docs_counts_path).exists():
    with open(kps_docs_counts_path, "wb") as fout:
        pickle.dump(kps_docs_counts, fout, pickle.HIGHEST_PROTOCOL)
        
kps_docs_ids_path = data_path + "/kps-docs-ids-simpseq10-nopost.pkl"
if not Path(kps_docs_ids_path).exists():
    with open(kps_docs_ids_path, "wb") as fout:
        pickle.dump(kps_docs_ids, fout, pickle.HIGHEST_PROTOCOL)

kps_directory_path = data_path + "/kps-directory-simpseq10-nopost.pkl"
if not Path(kps_directory_path).exists():
    with open(kps_directory_path, "wb") as fout:
        pickle.dump(kps_directory, fout, pickle.HIGHEST_PROTOCOL)
        
kps_directory_inverse_path = data_path + "/kps-directory-inverse-simpseq10-nopost.pkl"
if not Path(kps_directory_inverse_path).exists():
    with open(kps_directory_inverse_path, "wb") as fout:
        pickle.dump(kps_directory_inverse, fout, pickle.HIGHEST_PROTOCOL)

In [6]:
test_dataset_path = data_path + "/test-dataset.pkl"
if not Path(test_dataset_path).exists():
    with open(test_dataset_path, "wb") as fout:
        pickle.dump(test_dataset, fout, pickle.HIGHEST_PROTOCOL)

train_dataset_path = data_path + "/train-dataset.pkl"
if not Path(train_dataset_path).exists():
    with open(train_dataset_path, "wb") as fout:
        pickle.dump(train_dataset, fout, pickle.HIGHEST_PROTOCOL)